### 네이버 지식IN - 추석 선물

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm

- 지식인 추석 선물 검색결과 화면

In [2]:
url = f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page=1'
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [3]:
lis = soup.select('.basic1 > li')
len(lis)

10

- 답변 갯수가 5개 보다 큰 항목을 선택

In [5]:
li = lis[7]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=6130103&docId=454639453&qb=7LaU7ISdIOyEoOusvA==&enc=utf8&section=kin&rank=8&search_sort=0&spq=0'

In [6]:
driver.get(sub_url)
time.sleep(2)

In [9]:
# 더 보기 클릭 횟수 구하기
answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
click_count = (int(answer_num) - 1) // 5
answer_num, click_count

('8', 1)

In [10]:
# 더 보기 횟수 만큼 클릭하기
for _ in range(click_count):
    driver.find_element(By.ID, 'nextPageButton').click()
    time.sleep(2)

In [13]:
# 펼쳐진 화면을 BeautifulSoup으로 파싱
sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = sub_soup.select('.answer-content__list._answerList > div')
len(divs)

8

In [14]:
# 답변 목록에서 개별 답변을 선택하고 글을 추출하기
div = divs[0]
ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
len(ps)

45

In [22]:
answer = ''
for p in ps:
    text = p.get_text().strip()
    if text == '\u200b' or text == '':
        continue
    answer += p.get_text().strip() + '\n'
answer[:100]

'많은 인원수 준비 하실려면 부담이 크시겠어요\n연령대도 다양하시고\n남녀노소 가리지 않고 해드릴수 있는 선물을 추천 해드릴려고 합니다\n곧 풍성한 추석이 몇일 남지 않았어요\n부모님추석선'

In [27]:
# 답변 목록에 있는 모든 답변을 선택해서 글을 추출하기
answer_list = []
for div in divs:
    ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
    answer = ''
    for p in ps:
        text = p.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        answer += text + '\n'
    answer_list.append(answer)

In [28]:
len(answer_list)

8

In [29]:
driver.back()

- 한 화면에 있는 10개의 글에서 모든 답변을 추출하기

In [30]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)

    answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
    click_count = (int(answer_num) - 1) // 5
    print(click_count, end=' ')
    for _ in range(click_count):
        driver.find_element(By.ID, 'nextPageButton').click()
        time.sleep(2)

    sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs = sub_soup.select('.answer-content__list._answerList > div')
    for div in divs:
        ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
        answer = ''
        for p in ps:
            text = p.get_text().strip()
            if text == '\u200b' or text == '':
                continue
            answer += text + '\n'
        answer_list.append(answer)
    driver.back()
    time.sleep(2)

In [31]:
len(answer_list)

47

In [39]:
from tqdm import tqdm

In [40]:
url = f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page=1'
driver = webdriver.Chrome()
driver.get(url)

- 1 ~ 10 페이지에 대한 답변 가져오기

In [42]:
answer_list = []
driver = webdriver.Chrome()
for page in tqdm(range(1, 11)):
    url = f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page={page}'
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.basic1 > li')
    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
        click_count = (int(answer_num) - 1) // 5
        print(click_count, end=' ')
        for _ in range(click_count):
            driver.find_element(By.ID, 'nextPageButton').click()
            time.sleep(2)

        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = sub_soup.select('.answer-content__list._answerList > div')
        for div in divs:
            ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
            answer = ''
            for p in ps:
                text = p.get_text().strip()
                if text == '\u200b' or text == '':
                    continue
                answer += text + '\n'
            answer_list.append(answer)
        driver.back()
        time.sleep(3)
print()

  0%|          | 0/10 [00:00<?, ?it/s]

0 0 0 0 0 0 0 1 1 3 

 10%|█         | 1/10 [01:13<10:58, 73.13s/it]

1 5 0 1 1 1 3 3 0 0 

 20%|██        | 2/10 [02:47<11:24, 85.62s/it]

...

 80%|████████  | 8/10 [12:36<03:07, 93.89s/it]

3 0 2 0 1 2 0 2 8 0 

 90%|█████████ | 9/10 [14:17<01:36, 96.22s/it]

0 0 3 0 1 2 3 0 1 0 

100%|██████████| 10/10 [15:41<00:00, 94.11s/it]

In [43]:
len(answer_list)

994

- 파일에 쓰기

In [44]:
with open('data/추석선물.txt', 'w', encoding='UTF-8') as file:
    for answer in answer_list:
        file.write(answer + '\n')